In [2]:
import response
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 6000
max_len =  130
embedding_size = 128

(input_train, label_train), (input_test, label_test) = imdb.load_data(num_words=max_features)
input_train = pad_sequences(input_train, maxlen=max_len)
input_test = pad_sequences(input_test, maxlen=max_len)

'''
지시사항 1번
텐서 데이터의 차원을 1차원 늘리는 함수를 만들어 봅니다.
''' 

def expand_dim(_input):
    ## EX) (3,2) -> (3,2,1) 과 같이 뒤쪽에 차원을 늘림
    return tf.expand_dims(input=_input, axis=-1)
    
'''
지시사항 2~4번
2. tensorflow.keras를 활용하여 LSTM 레이어 층을 추가합니다.
3. 다음 층과의 계산을 위해 Lambda를 활용하여 출력의 차원을 늘려줍니다. 
4. Global Max Pooling 층을 추가 합니다.
''' 
def main():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_features, embedding_size),
        tf.keras.layers.LSTM(units=5),
        tf.keras.layers.Lambda(expand_dim),
        tf.keras.layers.GlobalMaxPool1D(),
        tf.keras.layers.Dense(20, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.05),      # 과적합 방지 
        tf.keras.layers.Dense(1, activation = tf.nn.sigmoid),
    ])

    model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['acc'])

    # 모델 학습하기
    model.fit(input_train, label_train, epochs=3, batch_size=64,verbose=2)

    # 테스트데이터 예측하기
    prediction = model.predict(input_test)
    print("각 예측의 결과는: ", prediction)
    # 예측 결과 binary로 변환하기
    y_pred = tf.round(prediction)
    print("각 예측의 결과를 binary로 표현하면: ", y_pred)

    # 예측결과 평가하기
    m = tf.keras.metrics.Accuracy()
    m.update_state(label_test, y_pred)
    print(m.result().numpy())

if __name__=='__main__':
    main()

Epoch 1/3
391/391 - 14s - loss: 0.5553 - acc: 0.7273
Epoch 2/3
391/391 - 12s - loss: 0.3945 - acc: 0.8403
Epoch 3/3
391/391 - 12s - loss: 0.3130 - acc: 0.8803
각 예측의 결과는:  [[0.14765629]
 [0.9588746 ]
 [0.7723073 ]
 ...
 [0.10790467]
 [0.08701646]
 [0.43011642]]
각 예측의 결과를 binary로 표현하면:  tf.Tensor(
[[0.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]], shape=(25000, 1), dtype=float32)
0.83748
